In [40]:
import pandas as pd
import numpy as np
import string
import os

In [41]:
for dirname, _, filenames in os.walk("D:\\Khoa hoc du lieu\\Anime Recommendations Database\\anime2020"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

D:\Khoa hoc du lieu\Anime Recommendations Database\anime2020\anime.csv
D:\Khoa hoc du lieu\Anime Recommendations Database\anime2020\animelist.csv
D:\Khoa hoc du lieu\Anime Recommendations Database\anime2020\anime_with_synopsis.csv
D:\Khoa hoc du lieu\Anime Recommendations Database\anime2020\rating_complete.csv
D:\Khoa hoc du lieu\Anime Recommendations Database\anime2020\watching_status.csv
D:\Khoa hoc du lieu\Anime Recommendations Database\anime2020\html folder\instructions.txt
D:\Khoa hoc du lieu\Anime Recommendations Database\anime2020\html folder\html\1\details.html
D:\Khoa hoc du lieu\Anime Recommendations Database\anime2020\html folder\html\1\pictures.html
D:\Khoa hoc du lieu\Anime Recommendations Database\anime2020\html folder\html\1\recomendations.html
D:\Khoa hoc du lieu\Anime Recommendations Database\anime2020\html folder\html\1\reviews_1.html
D:\Khoa hoc du lieu\Anime Recommendations Database\anime2020\html folder\html\1\reviews_10.html
D:\Khoa hoc du lieu\Anime Recommendatio

In [42]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [43]:
anime = pd.read_csv("D:\\VS Code Workspace\\Anime\\anime2020\\anime.csv")
anime = anime.reset_index()
anime.head()

,index,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",...,229170,182126,131625,62330,20688,8904,3184,1357,741,1580
1,1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,1-Sep-01,...,30043,49201,49505,22632,5805,1877,577,221,109,379
2,2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",...,50229,75651,86142,49432,15376,5838,1965,664,316,533
3,3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",...,2182,4806,10128,11618,5709,2920,1083,353,164,131
4,4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",...,312,529,1242,1713,1068,634,265,83,50,27


In [44]:
anime.columns

Index(['index', 'MAL_ID', 'Name', 'Score', 'Genres', 'English name',
       'Japanese name', 'Type', 'Episodes', 'Aired', 'Premiered', 'Producers',
       'Licensors', 'Studios', 'Source', 'Duration', 'Rating', 'Ranked',
       'Popularity', 'Members', 'Favorites', 'Watching', 'Completed',
       'On-Hold', 'Dropped', 'Plan to Watch', 'Score-10', 'Score-9', 'Score-8',
       'Score-7', 'Score-6', 'Score-5', 'Score-4', 'Score-3', 'Score-2',
       'Score-1'],
      dtype='object')

MAL_ID (int): Mã số của bộ phim trên trang web “My Anime List”

Name (text): Tên của phim

Score (float): Số điểm trung bình của phim trên thang điểm 10

Genres (text): Thể loại của bộ phim (Hài, Hành động, Lãng mạn, Kinh dị, …)

English name (text): Tên bộ phim theo tiếng Anh 

Japanese name (text): Tên bộ phim theo tiếng Nhật

Type (text): Hình thức công chiếu của phim (Chiếu TV, chiếu rạp, phần phim mở rộng,…)

Episodes (int): Số tập của phim

Aired (date/time): Thời gian công chiếu phim

Premiered (text): Mùa công chiếu phim (Mùa hè năm 1990, Mùa đông năm 2006,…)

Producers (text): Nhà sản xuất

Licensors (text): Đơn vị cấp phép công chiếu

Studios (text): Hãng phim

Source (text): Nguồn gốc của phim (Truyện tranh, tiểu thuyết, game,…)

Duration (text): Độ dài mỗi tập

Rating (text): Xếp hạng độ tuổi

Ranked (int): Xếp hạng của phim

Popularity (int): Độ phổ biến của phim, xếp hạng dựa vào số người dùng thêm vào danh sách xem của họ

Members (int): Số lượng thành viên cộng đồng nằm trong “nhóm” của bộ phim này

Favorites (int): Số người thêm bộ phim vào danh sách yêu thích

Watching (int): Số người đang xem phim

Completed (int): Só người đã xem xong bộ phim

On-Hold (int): Số người đang tạm dừng xem

Dropped (int): Số người đã bỏ xem phim

Plan to Watch (int): Số người lên kế hoạch xem phim

Score-10 (int): Số người đánh giá 10 cho phim

Score-9 (int): Số người đánh giá 9 cho phim

Score-8 (int): Số người đánh giá 8 cho phim

Score-7 (int): Số người đánh giá 7 cho phim

Score-6 (int): Số người đánh giá 6 cho phim

Score-5 (int): Số người đánh giá 5 cho phim

Score-4 (int): Số người đánh giá 4 cho phim

Score-3 (int): Số người đánh giá 3 cho phim

Score-2 (int): Số người đánh giá 2 cho phim

Score-1 (int): Số người đánh giá 1 cho phim

=========================================================================


Ta sẽ nhập tên một bộ phim. Mô hình sau đó sẽ xuất ra tên của một số bộ phim cùng thể loại với phim vừa nhập.

Kiểu dữ liệu của thuộc tính cần dự đoán: Gợi ý theo bản ghi

In [45]:
features = ['Score','Ranked','Studios', 'Genres', 'Episodes']

In [46]:
def combined_features(row):
    return str(row["Score"])+" "+ str(row["Ranked"])+" "+ str(row["Studios"])+" "+ str(row["Genres"])+" "+ str(row["Episodes"])+" "

def get_title_from_index(index):
    return anime[anime["index"] == index]["Name"].values[0]
def get_index_from_title(title):
    return anime[anime["Name"] == title]["index"].values[0]

In [47]:
anime["combined_feature"]=anime.apply(combined_features,axis=1)
anime["combined_feature"].head()

0    8.78 28 Sunrise Action, Adventure, Comedy, Dra...
1    8.39 159 Bones Action, Drama, Mystery, Sci-Fi,...
2    8.24 266 Madhouse Action, Sci-Fi, Adventure, C...
3    7.27 2481 Sunrise Action, Mystery, Police, Sup...
4    6.98 3710 Toei Animation Adventure, Fantasy, S...
Name: combined_feature, dtype: object

In [48]:
cv = CountVectorizer()
count_matrix=cv.fit_transform(anime["combined_feature"])

In [53]:
movie_liked_by_user = "One Punch Man"

In [54]:
cosine_sim = cosine_similarity(count_matrix)

In [56]:
liked_movie_index = cosine_sim[get_index_from_title(movie_liked_by_user)]
similar_anime = list(enumerate(liked_movie_index))
# similar_anime_sorted = sorted(similar_anime)
similar_anime.sort(key = lambda row: row[1],reverse=True)
# print(similar_anime)

for i in range(10):
    print(get_title_from_index(similar_anime[i][0]))

One Punch Man
One Punch Man: Road to Hero
One Punch Man Specials
Tokyo ESP
One Punch Man 2nd Season
One Punch Man 2nd Season Specials
One Punch Man 2nd Season Commemorative Special
Towa no Quon 4: Guren no Shoushin
Sumeba Miyako no Cosmos-sou Suttoko Taisen Dokkoida
Mekakucity Actors
